# Doing K fold validation for digits dataset
***SADGFIDSJIG***
:cat

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.datasets import load_digits

digits = load_digits()

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size = 0.3)

In [3]:
from sklearn import preprocessing

lr = LogisticRegression(solver='lbfgs', max_iter=10000)
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

0.9666666666666667

In [4]:
lr

LogisticRegression(max_iter=10000)

In [5]:
svm = SVC()
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

0.9888888888888889

In [6]:
rf = RandomForestClassifier(n_estimators = 40)
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9611111111111111

In [7]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 3)
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [8]:
numbers_range = list(range(1,20))
print(numbers_range[0:10])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [9]:
for train_index, test_index in kf.split(numbers_range):
    print(train_index, test_index)

[ 7  8  9 10 11 12 13 14 15 16 17 18] [0 1 2 3 4 5 6]
[ 0  1  2  3  4  5  6 13 14 15 16 17 18] [ 7  8  9 10 11 12]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12] [13 14 15 16 17 18]


### getScore function to run various models as input and get score as output

In [10]:
def getScore(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)
    

In [11]:
getScore(LogisticRegression(solver='lbfgs', max_iter=5000), X_train, X_test, y_train, y_test)

0.9666666666666667

In [12]:
getScore(SVC(), X_train, X_test, y_train, y_test)

0.9888888888888889

#### StratifiedKFold divides classification category in a uniform way for all folds like 3 classes dog cat horse . All three classes will be uniformly distributed among the folds


In [13]:
from sklearn.model_selection import StratifiedKFold

In [15]:
s_folds = StratifiedKFold(n_splits=3)

In [17]:
kf

KFold(n_splits=3, random_state=None, shuffle=False)

In [18]:
s_folds

StratifiedKFold(n_splits=3, random_state=None, shuffle=False)

In [24]:
scores_log_reg = []
scores_svm = []
scores_rf = []

for train_index, test_index in kf.split(digits.data):
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index],\
                                        digits.target[train_index], digits.target[test_index]
    #For multi class classification problem lbfgs solver is used
    scores_log_reg.append(getScore(LogisticRegression(solver='lbfgs', max_iter=5000),X_train, X_test, y_train, y_test))
    scores_svm.append(getScore(SVC(),X_train, X_test, y_train, y_test))
    scores_rf.append(getScore(RandomForestClassifier(n_estimators = 40),X_train, X_test, y_train, y_test))
    

In [25]:
print(scores_log_reg)
print(scores_svm)
print(scores_rf)

[0.9282136894824707, 0.9415692821368948, 0.9165275459098498]
[0.9666110183639399, 0.9816360601001669, 0.9549248747913188]
[0.9432387312186978, 0.9582637729549248, 0.9282136894824707]


### Mean of the scores

In [29]:
print(np.array(scores_log_reg).mean())
print(np.array(scores_svm).mean())
print(np.array(scores_rf).mean())

0.9287701725097385
0.9699499165275459
0.9387868670005565


### Working with the digits data

In [36]:
scores_log_reg_skf = []
scores_svm_skf = []
scores_rf_skf = []
for i, (train_index, test_index) in enumerate(s_folds.split(digits.data,digits.target)):
    X_train, X_test, y_train, y_test = digits.data[train_index], digits.data[test_index],\
                                        digits.target[train_index], digits.target[test_index]
    scores_log_reg_skf.append(getScore(LogisticRegression(solver='lbfgs', max_iter=5000),X_train, X_test, y_train, y_test))
    scores_svm_skf.append(getScore(SVC(),X_train, X_test, y_train, y_test))
    scores_rf_skf.append(getScore(RandomForestClassifier(n_estimators = 50),X_train, X_test, y_train, y_test))

In [37]:
print(scores_log_reg_skf)
print(scores_svm_skf)
print(scores_rf_skf)

[0.9248747913188647, 0.9382303839732888, 0.9232053422370617]
[0.9649415692821369, 0.9799666110183639, 0.9649415692821369]
[0.9248747913188647, 0.9532554257095158, 0.9315525876460768]


In [38]:
print(np.array(scores_log_reg_skf).mean())
print(np.array(scores_svm_skf).mean())
print(np.array(scores_rf_skf).mean())

0.9287701725097385
0.9699499165275459
0.9365609348914857


### Cross Validation and Hyper Parameter Tuning

In [39]:
from sklearn.model_selection import cross_val_score

In [42]:
#For multi class classification problem lbfgs solver is used
cross_val_score(LogisticRegression(solver='lbfgs', max_iter=5000), digits.data, digits.target)

array([0.925     , 0.87777778, 0.93871866, 0.93314763, 0.89693593])

In [43]:
cross_val_score(SVC(gamma='auto'), digits.data, digits.target)

array([0.41111111, 0.45      , 0.454039  , 0.44846797, 0.47910864])

In [50]:
rf1 = cross_val_score(RandomForestClassifier(n_estimators=60), digits.data, digits.target)
print(rf1)
print(" mean : ",rf1.mean())

[0.93333333 0.90833333 0.94986072 0.9637883  0.91922006]
 mean :  0.9349071494893222


In [51]:
rf2 = cross_val_score(RandomForestClassifier(n_estimators=60), digits.data, digits.target)
print(rf2)
print(" mean : ",rf2.mean())

[0.93611111 0.91111111 0.94150418 0.96935933 0.9275766 ]
 mean :  0.937132466728567


In [54]:
rf3 = cross_val_score(RandomForestClassifier(n_estimators=100,criterion="gini"),\
                                    digits.data, digits.target)
print(rf3)
print(" mean : ",rf3.mean())

[0.93888889 0.90555556 0.95264624 0.96657382 0.9275766 ]
 mean :  0.938248220365212
